## Noisy simulations 

Quantum noise can be characterised into coherent and incoherent sources of errors that arise during a computation. Coherent noise is commonly due to systematic errors originating from device miscalibrations, for example, gates implementing a rotation $\theta + \epsilon$ instead of $\theta$.

Incoherent noise has its origins in quantum states being entangled with the environment due to decoherence. This leads to mixed states which are probability distributions over pure states and are described by employing the density matrix formalism. 

We can model incoherent noise via quantum channels which are linear, completely positive, and trave preserving maps. The mathematical language used is of Kraus operators, $\{ K_i \}$, which satisfy the condition $\sum_{i} K_i^\dagger K_i = \mathbb{I}$. 

The bit-flip operation flips the qubit with probability $p$ and leaves it unchanged with probability $1-p$. This can be represented by employing Kraus operators: 


$K_0 = \sqrt{1-p} \begin{pmatrix}
  1 & 0 \\
  0 & 1
\end{pmatrix} $


$K_1 = \sqrt{p} \begin{pmatrix}
  0 & 1 \\
  1 & 0
\end{pmatrix} $

Lets implement the bit-flip channel using CUDA Quantum:

In [1]:
import cudaq
import numpy as np

# to model quantum noise, we need to utilise the density matrix simulator target
cudaq.set_target("density-matrix-cpu")

In [2]:
# Let's define a circuit

n_qubits = 2

kernel = cudaq.make_kernel()

qubits = kernel.qalloc(n_qubits)

kernel.x(qubits[0])
kernel.x(qubits[1])

In [3]:
# In the ideal noiseless case, we get 11 100% of the time as expected

ideal_counts = cudaq.sample(kernel, shots_count=1000)
ideal_counts.dump()

{ 11:1000 }


In [4]:
# You can build your own Kraus channels

p = 0.1  # probability of error

k0 = np.sqrt(1 - p) * np.array([[1.0, 0.0], [0.0, 1.0]], dtype=np.complex128)
k1 = np.sqrt(p) * np.array([[0.0, 1.0], [1.0, 0.0]], dtype=np.complex128)

# add the kraus operator to create a quantum channel
bitflip = cudaq.KrausChannel([k0, k1])

# you can also use built in noise channels
depol = cudaq.DepolarizationChannel(p)

# add the noise channels to an overall noise model
noise = cudaq.NoiseModel()

noise.add_channel(
    "x", [0],
    depol)  # apply the depol channel on the x operation on the 0th qubit
noise.add_channel(
    "x", [1],
    bitflip)  # apply the bitflip channel on the x operation on the 1st qubit

# we see unwanted results due to the effects of the noise channels
noisy_counts = cudaq.sample(kernel, noise_model=noise, shots_count=1000)
noisy_counts.dump()

{ 11:847 10:91 01:57 00:5 }
